# Data Cleaning and Preparation

Data preparation : loading, cleaning, transforming, rearranging

## 7.1 Handling Missing Data


In [3]:
import pandas as pd
import numpy as np

In [4]:
string_data = pd.Series(["aardvark", np.nan, None, "avocado"])

# Python's build in None value also been treated as NA
string_data.isna()

0    False
1     True
2     True
3    False
dtype: bool

In [6]:
num_data = pd.Series([0, 1, 3, 0])

num_data.isnull()

0    False
1    False
2    False
3    False
dtype: bool